In [1]:
# Import modules
import requests
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point

In [2]:
def fetch_data(base_url, dataset, api_key, num_records=99, offset=0):
    all_records = []
    max_offset = 9900
    while True:
        if offset > max_offset:
            break
        filters = f'{dataset}/records?limit={num_records}&offset={offset}'
        url = f'{base_url}{filters}&api_key={api_key}'
        try:
            result = requests.get(url, timeout = 10)
            result.raise_for_status()
            records = result.json().get('results')
        except requests.exceptions.RequestException as e:
            raise Exception(f'API request failed: {e}')
        if records is None:
            break
        all_records.extend(records)
        if len(records) < num_records:
            break
        offset += num_records
    df = pd.DataFrame(all_records)
    return df


# Base URL for API
API_KEY = os.environ.get('MELBOURNE_API_KEY', input('Please enter you API key: '))
BASE_URL = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'

# Load datasets using API
# Specific paths for the datasets
SPECIFIC_PATH_DEV = 'development-activity-monitor'
SPECIFIC_PATH_TREE_11 = 'tree-canopies-2011-urban-forest'
SPECIFIC_PATH_TREE_21 = 'tree-canopies-2021-urban-forest'

# Collect data using the API and the fetch_data function
dev_df = fetch_data(BASE_URL, SPECIFIC_PATH_DEV, API_KEY)
tree_11_df = fetch_data(BASE_URL, SPECIFIC_PATH_TREE_11, API_KEY)
tree_21_df = fetch_data(BASE_URL, SPECIFIC_PATH_TREE_21, API_KEY)


In [3]:
# Convert 'year_completed' to numeric
dev_df['year_completed'] = pd.to_numeric(dev_df['year_completed'])

# Filter developments that were completed on or after 2011 or are still in construction (i.e. is NaN).
dev_df = dev_df[(dev_df['year_completed'] >= 2011) | (dev_df['year_completed'].isna())]


In [4]:
tree_11_df

,geo_point_2d,geo_shape,tree_area,street_fro,data_lin_1,yearplant,street_nam,updated_by,shape_area,park_stree,...,street_to,family,ggis_id,overhead_c,date_plant,t11,canopy_dia,height_11,descriptio,roadseg_de
0,"{'lon': 144.91900753144674, 'lat': -37.7878123...","{'type': 'Feature', 'geometry': {'coordinates'...",3.4240327292,None,Tree Inventory: Existing data (fields from GIS...,0,None,Grace Detailed-GIS Services. info@gracegis.com.au,3.4240327292,None,...,None,None,67480,None,None,11,0.0,1.0,None,None
1,"{'lon': 144.9431510714485, 'lat': -37.78824563...","{'type': 'Feature', 'geometry': {'coordinates'...",14.0760594471,None,Tree Inventory: Existing data (fields from GIS...,0,None,Grace Detailed-GIS Services. info@gracegis.com.au,14.0760594471,None,...,None,None,67481,None,None,11,0.0,8.0,None,None
2,"{'lon': 144.944834333959, 'lat': -37.788266369...","{'type': 'Feature', 'geometry': {'coordinates'...",5.49496864441,None,Tree Inventory: Existing data (fields from GIS...,0,None,Grace Detailed-GIS Services. info@gracegis.com.au,5.49496864441,None,...,None,None,67483,None,None,11,0.0,2.0,None,None
3,"{'lon': 144.9053100082885, 'lat': -37.78756666...","{'type': 'Feature', 'geometry': {'coordinates'...",5.39040475601,None,Tree Inventory: Existing data (fields from GIS...,0,None,Grace Detailed-GIS Services. info@gracegis.com.au,5.39040475601,None,...,None,None,67491,None,None,11,0.0,3.0,None,None
4,"{'lon': 144.95489600858124, 'lat': -37.7884406...","{'type': 'Feature', 'geometry': {'coordinates'...",15.7932488746,None,Tree Inventory: Existing data (fields from GIS...,1998,None,Grace Detailed-GIS Services. info@gracegis.com.au,15.7932488746,Park,...,None,Myrtaceae,67495,None Present,1998-03-12T08:00:00.000Z,11,0.0,6.0,Tree - Eucalyptus camaldulensis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,"{'lon': 144.96871673703424, 'lat': -37.7932508...","{'type': 'Feature', 'geometry': {'coordinates'...",4.15527629006,None,Tree Inventory: Existing data (fields from GIS...,0,None,Grace Detailed-GIS Services. info@gracegis.com.au,4.15527629006,None,...,None,None,60123,None,None,11,0.0,2.0,None,None
9995,"{'lon': 144.93259426850673, 'lat': -37.7926158...","{'type': 'Feature', 'geometry': {'coordinates'...",2.59265241523,None,Tree Inventory: Existing data (fields from GIS...,0,None,Grace Detailed-GIS Services. info@gracegis.com.au,2.59265241523,None,...,None,None,60129,None,None,11,0.0,1.0,None,None
9996,"{'lon': 144.92412717173622, 'lat': -37.7924709...","{'type': 'Feature', 'geometry': {'coordinates'...",6.1555987244,None,Tree Inventory: Existing data (fields from GIS...,0,None,Grace Detailed-GIS Services. info@gracegis.com.au,6.1555987244,None,...,None,None,60133,None,None,11,0.0,2.0,None,None
9997,"{'lon': 144.92136564666146, 'lat': -37.7924266...","{'type': 'Feature', 'geometry': {'coordinates'...",21.1210662207,None,Tree Inventory: Existing data (fields from GIS...,2000,None,Grace Detailed-GIS Services. info@gracegis.com.au,21.1210662207,Street,...,None,Anacardiaceae,60137,None Present,2000-01-01T08:00:00.000Z,11,5.3,4.0,Tree - Schinus molle,None


In [5]:
# Filter to only include tree locations
tree_11_df = tree_11_df[['geo_point_2d']]
tree_21_df = tree_21_df[['geo_point_2d']]

In [6]:
# Filter to development completion and location
dev_df = dev_df.rename(columns={'geopoint': 'geo_point_2d'}) #for convenience in later function
dev_df = dev_df[['year_completed','geo_point_2d']]

In [7]:
# Function to convert 'geo_point_2d' from dictionary to separate latitude and longitude columns
def extract_lat_lon(df):
    df['latitude'] = df['geo_point_2d'].apply(lambda x: x.get('lat'))
    df['longitude'] = df['geo_point_2d'].apply(lambda x: x.get('lon'))
    return df

In [8]:
dev_df = extract_lat_lon(dev_df)
tree_11_df = extract_lat_lon(tree_11_df)
tree_21_df = extract_lat_lon(tree_21_df)

C:\Users\User\AppData\Local\Temp\ipykernel_2016\3263583511.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'] = df['geo_point_2d'].apply(lambda x: x.get('lat'))
C:\Users\User\AppData\Local\Temp\ipykernel_2016\3263583511.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'] = df['geo_point_2d'].apply(lambda x: x.get('lon'))


In [9]:
# Convert the DataFrame to a GeoDataFrame
def create_gdf(df):
    geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
    return gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

In [10]:
dev_gdf = create_gdf(dev_df)
tree_11_gdf = create_gdf(tree_11_df)
tree_21_gdf = create_gdf(tree_21_df)

In [11]:
# Perform spatial join between development sites and tree canopies
# Adjust 'radius' as necessary for your analysis
radius = 0.001  # roughly equal to 100 meters
buffer_dev_gdf = dev_gdf.copy()
buffer_dev_gdf['geometry'] = buffer_dev_gdf.geometry.buffer(radius)


C:\Users\User\AppData\Local\Temp\ipykernel_2016\378276997.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buffer_dev_gdf['geometry'] = buffer_dev_gdf.geometry.buffer(radius)


In [12]:
# Count trees around each development site for both years
trees_near_dev_2011 = gpd.sjoin(buffer_dev_gdf, tree_11_gdf, how='left', op='intersects')
trees_near_dev_2021 = gpd.sjoin(buffer_dev_gdf, tree_21_gdf, how='left', op='intersects')

# Group by development site and count trees
tree_counts_2011 = trees_near_dev_2011.groupby('development_site_id').size()
tree_counts_2021 = trees_near_dev_2021.groupby('development_site_id').size()

# Calculate the change in tree count
tree_change = tree_counts_2021 - tree_counts_2011

C:\Users\User\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3488: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3488: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


KeyError: 'development_site_id'